╔══<i><b>Alai-DeepLearning</b></i>════════════════════════════╗
###  &nbsp;&nbsp; **✎&nbsp;&nbsp;Week 8. Auto Encoder**
# Section 4. Auto Encoder

### _Objective_
1. Auto Encoder가 무엇인지 배워보도록 하겠습니다. <br>
2. 가장 기본적인 과소선형 Auto Encoder를 구현해보도록 하겠습니다.
  
╚═════════════════════════════════════════╝

In [0]:
%matplotlib inline

import tensorflow as tf
import math
from sklearn import datasets
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import os

### MNIST 가져오기

In [0]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("mnist_data/")

train_images = mnist.train.images
test_images = mnist.test.images
valid_images = mnist.validation.images

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting mnist_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting mnist_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


### [Optional.  Tensorflow Graph Visualization ]

---

> _Jupyter에서 Tensorflow에서 구성되는 Graph를 시각적으로 보여주기 위한 helper 메소드입니다._<br>

In [0]:
from IPython.display import clear_output, Image, display, HTML
import numpy as np    

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))

    display(HTML(iframe))

<br><br>

# \[ 1. Auto Encoder란? \]

----

----

> *오토 인코더는 딥러닝 모델 중 대표적인 unsupervised Learning 알고리즘입니다.*<br>
> *Input과 Output이 동일하도록 설계된 구조를 가집니다.* <br>

## 1. Auto Encoder의 구조
---

* 오토 인코더는 크게 Encoder, Bottleneck Code, Decoder로 구성됩니다.<br>

![](https://upload.wikimedia.org/wikipedia/commons/2/28/Autoencoder_structure.png)

* 오토인코더는 항등함수의 구조를 가집니다.<br>
$
x \approx decoder(encoder(x))\\
\approx decoder(z) \\
\approx \hat x
$<br>
encoder와 decoder를 통과했을 때, 원래의 값으로 복원할 수 있도록 만드는 구조입니다.


Auto Encoder는 크게 2가지의 모델을 함께 학습시킵니다.

* Encoder는 M개의 차원으로 이루어져 있는 입력값 X를 M보다 작은 N개의 차원으로 이루어진 code 값 Z로 변환하는 작업을 합니다.<br>
    보다 작은 차원으로 변환하기 때문에, 중요한 정보를 위주로 차원을 압축하는 것을 학습합니다.
    
* Decoder는 N개의 차원에서 실제의 데이터 셋인 M개의 차원으로 다시 재구성하는 작업을 합니다.<br>
    Decoder는 또 다른 말로, Generator, 즉 데이터를 생성하는 역할을 수행합니다.


## 2. Auto Encoder의 역할
---

* Auto Encoder는 딥러닝에서 매우 중요한 네트워크 구조 중 하나입니다.<br>
* 초기에 모델의 initialization에 많이 이용되었고, 현재는 매우 다양한 Variation을 보여줍니다.

#### 1) 차원 축소 (Encoder)
    - 데이터 내에서 핵심적인 정보만을 추출합니다.
    
#### 2) 노이즈 제거 (Encoder + Decoder)
    - 데이터 내 잡음을 Encoder를 통해 제거하고 핵심 정보를 추출하고, Decoder를 통해 복원합니다.
    
#### 3) 생성모델 (Decoder) 
    - Code에 따라 원하는 데이터를 생성합니다.

<br>
# \[2. AutoEncoder 텐서플로우로 구현하기\]
----

----

## 1. 선형 오토인코더 구현하기
---

선형 오토인코더는 일종의 PCA를 수행하는 것과 동일한 효과를 발휘합니다.<br>
이전까지 구현하였던 PCA는 분산이 가장 큰 방향(주성분)을 찾는다면, <br>
선형 AutoEncoder는 MSE를 최소화하는 방향으로 주성분, hidden Node를 구합니다.

### (1) Graph 구성하기

In [0]:
num_inputs = 784
num_hidden = 256

graph = tf.Graph()
with graph.as_default():
    x = tf.placeholder(tf.float32,(None, num_inputs), name='image')
    lr = tf.placeholder_with_default(0.01, None, name="learning_rate")
    
    # encoder 을 구현합니다.
    with tf.variable_scope('encoding'):
        #fix me# 
    hidden = tf.identity(hidden, name="hidden")
    # dencoder 을 구현합니다.
    with tf.variable_scope('decoding'):
        #fix me# 
        
    outputs = tf.identity(outputs, name="reconstruction")

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
show_graph(graph)

### (2) 손실함수 및 평가지표 구성하기

In [0]:
with graph.as_default():
    with tf.variable_scope("loss"):
        # mean square error  
        mse = #fix me#

        # 변수들을 불러옵니다. 
        weights = graph.get_collection(#fix me#)
        loss = mse
    
    with tf.variable_scope("metric"):
        # rmse을 이용해 목적함수와 예측값이 얼만큼 차이가 나는지 확인합니다. 
        rmse = tf.sqrt(mse, name="rmse")
    # adam optimizer 을 구현합니다.
    #fix me #

Instructions for updating:
Use tf.cast instead.


In [0]:
show_graph(graph)

### (3) Graph 학습시키기

In [0]:
num_epoch = 10 # epoch 횟수
num_batch = 128 # 배치 크기
num_data = mnist.train.num_examples # data의 수
num_step = num_data // num_batch # 1 epoch 별 학습 횟수

with graph.as_default():
    sess = tf.Session(graph=graph)
    
    sess.run(tf.global_variables_initializer())
    for i in range(num_epoch):        
        for _ in range(num_step):
            images, _ = mnist.train.next_batch(num_batch)
            # training 
            sess.run(#fix me #, feed_dict={x:images,lr:0.0005})
        # 1 epoch 가 돌때마다 정답과 예측의 차이를 확인합니다        
        rmse_value = sess.run(#fix me#, feed_dict={x:valid_images})
        print("{:2d}th epoch validation RMSE : {:.5f}".format(i
                                                              
                                                              
                                                              
                                                              ,rmse_value))

 0th epoch test RMSE : 0.08915
 1th epoch test RMSE : 0.06662
 2th epoch test RMSE : 0.05644
 3th epoch test RMSE : 0.05032
 4th epoch test RMSE : 0.04632
 5th epoch test RMSE : 0.04358
 6th epoch test RMSE : 0.04176
 7th epoch test RMSE : 0.04062
 8th epoch test RMSE : 0.03988
 9th epoch test RMSE : 0.03937


### (4) 결과 확인하기

재구축된 이미지와 원래 이미지를 비교해 보도록 하겠습니다.

In [0]:
reconstruction = sess.run(outputs,
                          feed_dict={x:test_images})
reconstruction = np.clip(reconstruction,0.,1.)
reconstruction = reconstruction.reshape(-1,28,28)

fig, (ax1, ax2) = plt.subplots(2,10, figsize=(16,5))

for i in range(10):
    ax = ax1[i]
    ax.set_xticks([])
    ax.set_yticks([])    
    ax.imshow(test_images[i].reshape(28,28))
    
for i in range(10):
    ax = ax2[i]
    ax.set_xticks([])
    ax.set_yticks([])    
    ax.imshow(reconstruction[i])

plt.show()

<br>

# 2. Hidden Node 수에 따른 오차 변화
---

PCA와 마찬가지로, encoding 단계에서 Hidden Node의 수에 따라 성능에 큰 차이를 보입니다.

### (1) graph 메소드 구현하기

In [0]:
def build_autoencoder(num_hidden, num_inputs=784):
    graph = tf.Graph()
    # fix me #

In [0]:
graph = build_autoencoder(10)
show_graph(graph)

### (2) 모델 학습시키기

hidden수에 따라 다양하게 모델을 학습시켜 평가해보도록 하겠습니다.

In [0]:
num_epoch = 10 # epoch 횟수
num_batch = 128 # 배치 크기
num_data = mnist.train.num_examples # data의 수
num_step = num_data // num_batch # 1 epoch 별 학습 횟수

test_images = mnist.test.images

for num_hidden in [2, 50, 100, 200]:
    # fix me #

#  

---

    Copyright(c) 2019 by Public AI. All rights reserved.<br>
    Writen by PAI, SangJae Kang ( rocketgrowthsj@publicai.co.kr )  last updated on 2019/04/14

---